Decision Tree workflow/process 
    Select feature
    Sort data
    Compute mid point or threshold value 
    Split (feature based on threshold)
    Compute fid3
    Compute entropy
    Compute weighted entropy
    Repeate for all threshold values
    Repeate for all features
    Find the weighted entropy with least value => best split 




In [1]:
#Import python libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier


In [2]:
#Create a dummy dataset
array_x = np.array([[3,7],[1,8],[4,5],[2,6]])
print(array_x)

array_y = np.array([1, 0, 1, 0])
print(array_y)

[[3 7]
 [1 8]
 [4 5]
 [2 6]]
[1 0 1 0]


In [ ]:
#Class Node (Root node, internal/decision node and leaf node)
class Node:
    def __init__(self,  feature = None, threshold = None, left = None, right = None, value = None):
        self.feature  = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value 

    def is_leaf_node(self):
        return self.value is not None
    

In [ ]:
#Common methods in decision trees
def most_common_label(y):
    labels, counts = np.unique(y, return_count = True)
    return labels[np.argmax(counts)]

def split(x_column, split_threshold):
    left_node = np.argwhere(x_column <= split_threshold).flatten
    right_node = np.argwhere(x_column > split_threshold).flatten
    return left_node, right_node;

def entropy(y):
    fid3 = np.mean(y)
    if fid3 == 0 or fid3 == 1:
        return 0
    
    return -fid3 * np.log(fid3) - (1 - fid3) * np.log(1 - fid3)

def information_gain(x_column, y, split_threshold):
    left_node, right_node = split(x_column, split_threshold)
    if left_node == 0 or right_node == 0:
        return 0
    
    p_left = len(left_node)/len(y)
    p_right = len(right_node)/len(y)
    weighted_entropy = p_left * entropy(left_node) + p_right * entropy(right_node)
    return weighted_entropy
